In [1]:
import os

In [2]:
%pwd

'c:\\Users\\cjx14\\Course_Krish_Naik\\End-to-End-Machine-Learning-Project-with-MLOPs-Flow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\cjx14\\Course_Krish_Naik\\End-to-End-Machine-Learning-Project-with-MLOPs-Flow'

### Setup Dagshub Environment

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Kanon14/End-to-End-Machine-Learning-Project-with-MLOPs-Flow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Kanon14"
os.environ["MLFLOW_TRACKING_PASSWORD"]="2a57f710b92c4261b14ebc77457c63fc1b2320f4"

### Define Config Entity

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

### Prepare Configuration Manager

In [15]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/Kanon14/End-to-End-Machine-Learning-Project-with-MLOPs-Flow.mlflow"
        )
        
        return model_evaluation_config

### Components

In [17]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib

In [18]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        text_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme
        
        with mlflow.start_run(nested=True):
            
            predicted_quantities = model.predict(text_x)
            
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_quantities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case.
                # Please refer to the doc for more information.
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
            
            

In [19]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-04-05 16:43:12,120: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-05 16:43:12,121: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-05 16:43:12,124: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-05 16:43:12,125: INFO: common: Creating directory: artifacts]
[2024-04-05 16:43:12,126: INFO: common: Creating directory: artifacts/model_evaluation]
[2024-04-05 16:43:13,003: INFO: common: JSON file: artifacts\model_evaluation\metrics.json saved successfully]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/04/05 16:43:20 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
